In [5]:
import tensorflow as tf
from keras.layers import Input, Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D, concatenate, Activation
from keras.applications.mobilenet import MobileNet as Net

import numpy as np
import pandas as pd
import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Flatten
from keras.layers import Embedding
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, model_from_json
from keras.layers import Dense, LSTM, Dropout, Bidirectional, merge
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from IPython.display import SVG
#from keras.utils.visualize_util import model_to_dot
from nltk.corpus import stopwords
import wikipedia
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool

from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.layers import Input, merge
from keras.optimizers import Adam
import cv2
import numpy as np
import os
from tqdm import tqdm
from keras.models import Sequential

from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

ALPHA = 0.3  # Triplet Loss Parameter

from keras.layers import Input,Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D,concatenate,Activation
from keras.applications.xception import Xception as Net
from keras.preprocessing import image
from keras.applications.xception import preprocess_input
from keras.models import Model

from keras import backend as K
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, merge
from keras.optimizers import Adam

#import data
#import metrics

In [8]:
#Train Data
df_train = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Microsoft/data/data.tsv', 
                       delimiter="\t",header=None)

In [9]:
#Test Data
df_test = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Microsoft/data/eval1_unlabelled.tsv', 
                      delimiter="\t",header=None)

In [10]:
#Column names Train
df_train.columns = ["id", "question", "answer", "is_ans", "seq_no"]

In [11]:
#Column names Test
df_test.columns = ["id", "question", "answer", "seq_no"]

In [12]:
q1_train = list(map(str, df_train['question'].tolist()))
q2_train = list(map(str, df_train['answer'].tolist()))
is_ans = df_train['is_ans'].tolist()

In [13]:
q1_train = q1_train[:100000]
q2_train = q2_train[:100000]
is_ans = is_ans[:100000]

In [15]:
#length of the seq
MAX_SEQUENCE_LENGTH = 200
#For tokenizer how many words needed
MAX_NB_WORDS = 100000
#Dimension of the embed matrix
EMBEDDING_DIM = 50
#validation split for model
VALIDATION_SPLIT = 0.01

In [16]:
#Tokenize the documnent into tokens for each word in doc
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(q1_train + q2_train)

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/keras/preprocessing/text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [17]:
folder_path= '/Users/s0c02nj/Downloads/glove.6B/glove.6B.50d.txt'
f=open(folder_path)
doc=f.readlines()
#****VIP
#**WORD TO VEC DICTIONARY
#Forming a dictionary-word2vec
word2vec={}
key=[]
#looping though the doc.in the doc the entire thing is saved and is separated by a space bar.
for line in doc:
    #parts contains every word separately for doc1
    parts=line.split(' ')
    #part[0] contains the word
    word=parts[0]
    key.append(word)
    #embed contains the vector
    embed=np.array(parts[1:],dtype='float32')
    #filling up the dictionary
    word2vec[word]=embed

In [18]:
print('Found %s word vectors.' % len(word2vec))

Found 400000 word vectors.


In [19]:
#getting the tokenized seq for the docs
q1_seq = tokenizer.texts_to_sequences(q1_train)
q2_seq = tokenizer.texts_to_sequences(q2_train)

word_index = tokenizer.word_index
#print('Found %s unique tokens.' % len(word_index))

In [35]:
#len(q1_seq[150])

In [36]:
q1_seq = pad_sequences(q1_seq, maxlen=10, padding='post')
q2_seq = pad_sequences(q2_seq, maxlen=70, padding='post')

In [37]:
labels = np.array(is_ans)

In [ ]:
#Embedding matrix creation
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= nb_words:
        continue
    if word in word2vec:
        embedding_vector = word2vec[word]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))